In [ ]:
# Try out some data augmentation to make it better.
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from keras.optimizers import Adam


Functions to clean up the work flow

In [ ]:
# Loading in data
def load_data(file_path="FashionMNIST/data/fashion_mnist_scaled.npz"):
    data = np.load(file_path, allow_pickle=True)
    X_train, y_train, X_test, y_test = [data[file] for file in data.files]
    return X_train, y_train, X_test, y_test


# Getting an augmented dataset
def augmenting_data(gen, X, y, num_augmented_imgs=5):
    aug_iter = gen.flow(X, y, batch_size=len(X))
    data = [next(aug_iter) for num in range(num_augmented_imgs)]
    y1 = [data[num][1] for num in range(num_augmented_imgs)]
    x = [data[num][0] for num in range(num_augmented_imgs)]
    return np.concatenate([np.vstack(x), X]), np.concatenate([np.vstack(y1).reshape(-1), y])


In [ ]:
# Loading in data
X_train, y_train, X_test, y_test = load_data()


In [ ]:
# Creating image generator
gen = ImageDataGenerator(rotation_range=10, width_shift_range=.1, zoom_range=.1,
                         horizontal_flip=True)


In [ ]:
# Creating augmented dataset
X_train_augmented, y_train_augmented = augmenting_data(gen, X_train, y_train, 2)


In [ ]:
# Callbacks for training
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=1e-4, mode='min')
callbacks = [es, mcp_save, reduce_lr_loss]


In [ ]:
# Making a deeper model architecture
model2 = Sequential([
    Conv2D(120, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(),

    Conv2D(60, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),

    Conv2D(30, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(10, activation='softmax')
])


In [ ]:
# Compiling the simple model
model2.compile(Adam(.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Fitting model with specifying validation set
model2.fit(X_train_augmented, y_train_augmented, validation_split=.1, epochs=100,
           callbacks=callbacks, verbose=2)